In [2]:
!pip install bitsandbytes
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 774.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [3]:
from huggingface_hub import login
#login(token="xxx", add_to_git_credential=True)

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [5]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [6]:
# Load the tokenizer and model with the token

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=quantization_config, 
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
df = pd.read_csv('/kaggle/input/sample.csv')
df.head()

,id,author_id,in_reply_to_user_id,quoted_user_id,retweeted_user_id,text,public_metrics.like_count,public_metrics.quote_count,public_metrics.reply_count,public_metrics.retweet_count,...,link_text,party,name,Occupation(s),Education,Residence[5],gpt_sentiment,age,gender,bert_sentiment
0,1174499599181959168,262192574,NaN,NaN,NaN,We should never forget the sacrifices made by ...,22,0,0,2,...,MISSISSIPPI (WLOX) - U.S. Senator Cindy Hyde-S...,Republican,Cindy Hyde-Smith,Lobbyist Farmer,Copiah-Lincoln Community College (AA) Universi...,Brookhaven,positive,65,female,positive
1,1320738770543628288,15808765,NaN,NaN,NaN,It's no secret what Donald Trump and Senate Re...,1494,67,264,323,...,President Trump says in an interview with “60 ...,Democrat,Cory Booker,Lawyer,"Stanford University (BA, MA) Queen's College, ...",Newark,negative,55,male,negative
2,998614598424068101,15745368,NaN,NaN,NaN,Some #Florida elections officials didn’t like ...,259,19,105,112,...,Comment on this story Comment Gift Article Sha...,Republican,Marco Rubio,Lawyer,University of Florida (BA) University of Miami...,West Miami[13],neutral,53,male,negative
3,955084959840129024,13218102,NaN,NaN,NaN,"One woman, two marches: Can women who oppose a...",14,7,61,9,...,Rachel Lamb saw as much fate as coincidence in...,Republican,John Cornyn,Lawyer,"Trinity University (BA) St. Mary's University,...",Austin,neutral,72,male,neutral
4,1255486188892422149,1080960924687704064,NaN,NaN,NaN,The coronavirus pandemic continues to highligh...,8,1,1,2,...,WASHINGTON D.C. (KFVS) - U.S. Senators Josh Ha...,Republican,Josh Hawley,Lawyer Professor,Stanford University (BA) Yale University (JD),Ozark,neutral,44,male,negative


In [8]:
# function with a prompt example 
def make_prompt(party, age, news_content): 
    prompt = (
        f"You are a politician arguing on Twitter and a representative of the {party} party. You are {age} years old."
        f"Read the article below and write a tweet with no more than 280 characters to comment on it.\n\n"
        f"Article:\n{news_content}"
    )
    return prompt

In [9]:
prompts = [
    make_prompt(row["party"], row["age"], row['link_text'])
    for _, row in df.iterrows()
]

In [10]:
# Check and set the padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize the model's embeddings to include the new token

In [17]:
responses = []

In [ ]:
for prompt in tqdm(prompts[5356:]):

    tokens = tokenizer.encode(prompt)
    if len(tokens) > 8192:
        tokens = tokens[:8192]
        prompt = tokenizer.decode(tokens)

    model_inputs = tokenizer([prompt], return_tensors="pt").to('cuda')
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, top_k=10, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].replace(prompt, "")
    responses.append(answer)
 
    # remove inputs from GPU
    del model_inputs
    torch.cuda.empty_cache()

  8%|▊         | 391/4644 [48:11<8:20:12,  7.06s/it] 

In [ ]:
len(responses)

In [14]:
# Create a DataFrame from the list
responses_df = pd.DataFrame(responses, columns=["mistral"])

# Save the DataFrame as a CSV file
responses_df.to_csv("mistral_age_5356.csv", index=False)